Esse notebook será utilizado para comparar as bases finais geradas pelo ETL e pelo ELT.

In [ ]:
# Vamos fazer os imports necessários

import pandas as pd
import sqlite3
import numpy as np
import os

In [ ]:
# Conectar ao Data Warehouse criado pelo ELT
db_path = 'samu_dw.db'
if not os.path.exists(db_path):
    print("❌ ERRO: Arquivo samu_dw.db não encontrado. Rode o notebook ELT_SAMU.ipynb primeiro.")
else:
    conn = sqlite3.connect(db_path)
    print("✅ Conectado ao Banco de Dados.")

In [ ]:
# query para descontruir o star schema refazendo a tabela plana
query_sql = """
SELECT 
    f.id_atendimento,
    f.data as data,
    f.hora_minuto,
    l.municipio,
    l.bairro,
    m.tipo,
    m.subtipo,
    m.motivo_desfecho_limpo as motivo_desfecho,
    f.sexo,
    f.idade,
    f.ano_origem
FROM fato_atendimentos f
LEFT JOIN dim_local l ON f.id_local = l.id_local
LEFT JOIN dim_motivo m ON f.id_motivo = m.id_motivo
"""
#carregando o banco para Pandas 
df_elt = pd.read_sql(query_sql, conn)

# Ajuste de tipos para bater com o Pandas
df_elt['idade'] = pd.to_numeric(df_elt['idade'])
df_elt['ano_origem'] = df_elt['ano_origem'].astype(int)

In [ ]:
# Definição das colunas padrão
colunas_padrao = [
    '_id', 'data', 'hora_minuto', 'municipio', 'bairro', 'endereco',
    'origem_chamado', 'tipo', 'subtipo', 'sexo', 'idade',
    'motivo_finalizacao', 'motivo_desfecho'
]

try:
    # 1. Leitura dos CSVs 
    df_23 = pd.read_csv('../data/samu_2023.csv', header=None, names=colunas_padrao, low_memory=False)
    df_24 = pd.read_csv('../data/samu_2024.csv', header=None, names=colunas_padrao, low_memory=False)
    df_25 = pd.read_csv('../data/samu_2025.csv', header=None, names=colunas_padrao, low_memory=False)
    
    # Adicionar ano de origem
    df_23['ano_origem'] = 2023
    df_24['ano_origem'] = 2024
    df_25['ano_origem'] = 2025
    
    df_etl = pd.concat([df_23, df_24, df_25])
    
    # --- REPLICAÇÃO DAS REGRAS DE NEGÓCIO ---
    
    # 2. Filtrar "lixo" (cabeçalhos repetidos no meio do arquivo)
    df_etl = df_etl[df_etl['idade'].astype(str).str.lower() != 'idade'].copy()
    
    # 3. Tratamento de Idade (Converter para numérico e preencher com mediana)
    df_etl['idade'] = pd.to_numeric(df_etl['idade'], errors='coerce')
    mediana = df_etl['idade'].median()
    df_etl['idade'] = df_etl['idade'].fillna(mediana)
    
    # 4. Normalização de Texto (UPPER) - Igual ao SQL dim_local/dim_motivo
    cols_upper = ['municipio', 'bairro', 'tipo', 'subtipo', 'sexo']
    for col in cols_upper:
        df_etl[col] = df_etl[col].astype(str).str.upper()
    
    # 5. Regra específica do Motivo Desfecho (limpar prefixos numéricos como "1. ", "2. ")
    def limpar_motivo(val):
        val = str(val).upper()
        if len(val) > 3 and val[0].isdigit() and val[1] == '.':
            return val[3:].strip() # Remove "1. " e espaços
        return val
        
    df_etl['motivo_desfecho'] = df_etl['motivo_desfecho'].apply(limpar_motivo)
    
    # 6. Tratar Nulos do Subtipo (SQL usa COALESCE para 'NAO INFORMADO')
    df_etl['subtipo'] = df_etl['subtipo'].replace('NAN', 'NAO INFORMADO')

    # Renomear coluna de ID para bater com o SQL
    df_etl = df_etl.rename(columns={'_id': 'id_atendimento'})
    
    # Selecionar apenas as colunas que existem no SQL para comparar
    colunas_finais = [
        'id_atendimento', 'data', 'hora_minuto', 'municipio', 'bairro', 
        'tipo', 'subtipo', 'motivo_desfecho', 'sexo', 'idade', 'ano_origem'
    ]
    df_etl = df_etl[colunas_finais]

    print(f"✅ Dados processados via Pandas (ETL): {df_etl.shape[0]} linhas.")

except Exception as e:
    print(f"❌ Erro ao processar CSVs: {e}")

In [ ]:
# Ordenar ambos os DataFrames para garantir que as linhas estejam na mesma ordem
# Ordenamos por ID e Ano para garantir unicidade
df_elt = df_elt.sort_values(by=['id_atendimento', 'ano_origem']).reset_index(drop=True)
df_etl = df_etl.sort_values(by=['id_atendimento', 'ano_origem']).reset_index(drop=True)

print("\n--- RELATÓRIO DE VALIDAÇÃO ---")
print(f"Total Linhas SQL (ELT):    {len(df_elt)}")
print(f"Total Linhas Pandas (ETL): {len(df_etl)}")

try:
    # check_dtype=False é usado porque o SQLite pode retornar floats com precisão levemente diferente
    # ou strings como objetos, mas o conteúdo deve ser igual.
    pd.testing.assert_frame_equal(df_etl, df_elt, check_dtype=False, atol=0.01)
    
    print("\n🏆 SUCESSO ABSOLUTO!")
    print("A base gerada pelo ELT (SQL) é matematicamente IDÊNTICA à base gerada via Pandas (ETL).")
    print("O Data Warehouse está validado e confiável.")
    
except AssertionError as e:
    print("\n⚠️ DIVERGÊNCIA ENCONTRADA!")
    print("As bases não são iguais. Veja o detalhe do erro abaixo:")
    print(e)